## borramos collection en mongoDB

In [1]:
from pymongo import MongoClient
#mongodb conection
client = MongoClient('localhost', 27017)
db = client.bici
collection = db.datacleaned
collection.drop()

# importamos los datos de BICIMAD
<img src="https://pbs.twimg.com/profile_images/621342633655271424/26fdqUsf.png" style="width:150px; display:inline">


In [2]:
# from urllib import urlopen
import urllib.request

import simplejson as json

#Es un api no oficial pero que ofrece datos cada 10 min

url = urllib.request.urlopen('http://bicimad-api.herokuapp.com/api-v1/locations/').read()
url = json.loads(url)

In [3]:
import pandas as pd
dataPre = url["locations"]
data_mad = pd.DataFrame(dataPre)

In [4]:
data_mad['servicio']  = 'bicimad'

In [5]:
data_mad.head()

,activo,direccion,idestacion,latitud,libres,longitud,luz,nombre,numero_estacion,porcentaje,servicio
0,True,Puerta del Sol No 1,1a,40.4168961,12,-3.7024255,False,Puerta del Sol A,1a,50.000000,bicimad
1,True,Calle Fuencarral No 108,5,40.4285280,21,-3.7020599,False,Fuencarral,5,77.777778,bicimad
2,True,Calle Marques de la Ensenada No 16,10,40.4250863,13,-3.6918807,False,Marques de la Ensenada,10,54.166667,bicimad
3,True,Calle San Vicente Ferrer No 64,15,40.4261649,15,-3.7073764,False,S.Vicente Ferrer,15,71.428571,bicimad
4,True,Calle Alcala No 49,20a,40.4192342,12,-3.6954615,False,Banco de España A,20a,50.000000,bicimad


# importamos los datos de BICING
<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/a/a1/Bicing_logo.svg/200px-Bicing_logo.svg.png" style="width:150px; display:inline">


In [14]:
url_bcn = urllib.request.urlopen('http://wservice.viabicing.cat/v2/stations').read()
url_bcn = url_bcn.split(b'\n')
# url_bcn = url_bcn[1:len(url_bcn)]
url_bcn = eval(url_bcn[0])
url_bcn = url_bcn["stations"]
print((url_bcn))
data_bcn = pd.DataFrame(url_bcn)

2
[{'status': 'OPN', 'slots': '2', 'streetName': 'Gran Via Corts Catalanes', 'nearbyStations': '24, 369, 387, 426', 'longitude': '2.180042', 'id': '1', 'altitude': '21', 'latitude': '41.397952', 'type': 'BIKE', 'streetNumber': '760', 'bikes': '20'}, {'status': 'OPN', 'slots': '15', 'streetName': 'Roger de Flor\\/ Gran Vía', 'nearbyStations': '360, 368, 387, 414', 'longitude': '2.17706', 'id': '2', 'altitude': '21', 'latitude': '41.39553', 'type': 'BIKE', 'streetNumber': '126', 'bikes': '9'}, {'status': 'OPN', 'slots': '16', 'streetName': 'Ali Bei', 'nearbyStations': '4, 6, 119, 419', 'longitude': '2.181137', 'id': '3', 'altitude': '21', 'latitude': '41.393699', 'type': 'BIKE', 'streetNumber': '44', 'bikes': '7'}, {'status': 'OPN', 'slots': '18', 'streetName': 'Ribes', 'nearbyStations': '3, 5, 359, 419', 'longitude': '2.18149', 'id': '4', 'altitude': '21', 'latitude': '41.39347', 'type': 'BIKE', 'streetNumber': '13', 'bikes': '3'}, {'status': 'OPN', 'slots': '14', 'streetName': 'Pg Lluí

In [15]:
data_bcn['servicio']  = 'bicing'

In [16]:
data_bcn['direccion'] = data_bcn["streetName"] + data_bcn["streetNumber"]
data_bcn['slots'] = data_bcn["bikes"].astype(float) + data_bcn["slots"].astype(float)
data_bcn['porcenjate'] = data_bcn['bikes'].astype(float) / data_bcn['slots'].astype(float)


del data_bcn["streetName"]
del data_bcn["streetNumber"]
del data_bcn["nearbyStations"]
del data_bcn["altitude"]
del data_bcn["type"]

In [17]:
del data_mad["luz"]
data_mad['slots'] = data_mad['libres'] / data_mad['porcentaje'] *100
del data_mad['porcentaje']
data_mad['porcentaje'] = data_mad['libres'].astype(float) / data_mad['slots'].astype(float)
del data_mad['numero_estacion']
del data_mad['nombre']

In [18]:
data_bcn.columns 

Index(['bikes', 'id', 'latitude', 'longitude', 'slots', 'status', 'servicio',
       'direccion', 'porcenjate'],
      dtype='object')

In [19]:
data_mad.columns

Index(['activo', 'direccion', 'idestacion', 'latitud', 'libres', 'longitud',
       'servicio', 'slots', 'porcentaje'],
      dtype='object')

In [21]:
data_bcn.head()

,bikes,id,latitude,longitude,slots,status,servicio,direccion,porcenjate
0,20,1,41.397952,2.180042,22.0,OPN,bicing,Gran Via Corts Catalanes760,0.909091
1,9,2,41.39553,2.17706,24.0,OPN,bicing,Roger de Flor\/ Gran Vía126,0.375000
2,7,3,41.393699,2.181137,23.0,OPN,bicing,Ali Bei44,0.304348
3,3,4,41.39347,2.18149,21.0,OPN,bicing,Ribes13,0.142857
4,22,5,41.391075,2.180223,36.0,OPN,bicing,Pg Lluís Companys11,0.611111


In [22]:
data_bcn.columns = ['libres', 'id_estacion', 'latitud', 'longitud', 'slots', 'status', 'servicio', 'direccion','porcentaje']
data_mad.columns = ['status', 'direccion', 'id_estacion', 'latitud', 'libres', 'longitud','servicio', 'slots', 'porcentaje']

In [23]:
data_mad=data_mad.dropna() 
data_bcn=data_bcn.dropna() 

In [24]:
import warnings
warnings.filterwarnings("ignore")

In [25]:
data_mad['disponibilidad'] = 'baja disponibilidad'
data_mad['disponibilidad'][(data_mad['porcentaje'] > 0.20) & (data_mad['porcentaje'] <= 0.5)] = 'disponibilidad media'
data_mad['disponibilidad'][(data_mad['porcentaje'] > 0.50) & (data_mad['porcentaje'] <= 0.8)] = 'alta disponibilidad'
data_mad['disponibilidad'][data_mad['porcentaje'] >0.80] = 'muy alta disponibilidad'


In [26]:
data_bcn['disponibilidad'] = 'baja disponibilidad'
data_bcn['disponibilidad'][(data_bcn['porcentaje'] > 0.20) & (data_bcn['porcentaje'] <= 0.5)] = 'disponibilidad media'
data_bcn['disponibilidad'][(data_bcn['porcentaje'] > 0.50) & (data_bcn['porcentaje'] <= 0.8)] = 'alta disponibilidad'
data_bcn['disponibilidad'][data_bcn['porcentaje'] >0.80] = 'muy alta disponibilidad'



In [27]:
data_mad.head()

,status,direccion,id_estacion,latitud,libres,longitud,servicio,slots,porcentaje,disponibilidad
0,True,Puerta del Sol No 1,1a,40.4168961,12,-3.7024255,bicimad,24.0,0.500000,disponibilidad media
1,True,Calle Fuencarral No 108,5,40.4285280,21,-3.7020599,bicimad,27.0,0.777778,alta disponibilidad
2,True,Calle Marques de la Ensenada No 16,10,40.4250863,13,-3.6918807,bicimad,24.0,0.541667,alta disponibilidad
3,True,Calle San Vicente Ferrer No 64,15,40.4261649,15,-3.7073764,bicimad,21.0,0.714286,alta disponibilidad
4,True,Calle Alcala No 49,20a,40.4192342,12,-3.6954615,bicimad,24.0,0.500000,disponibilidad media


In [28]:
data_bcn.head()

,libres,id_estacion,latitud,longitud,slots,status,servicio,direccion,porcentaje,disponibilidad
0,20,1,41.397952,2.180042,22.0,OPN,bicing,Gran Via Corts Catalanes760,0.909091,muy alta disponibilidad
1,9,2,41.39553,2.17706,24.0,OPN,bicing,Roger de Flor\/ Gran Vía126,0.375000,disponibilidad media
2,7,3,41.393699,2.181137,23.0,OPN,bicing,Ali Bei44,0.304348,disponibilidad media
3,3,4,41.39347,2.18149,21.0,OPN,bicing,Ribes13,0.142857,baja disponibilidad
4,22,5,41.391075,2.180223,36.0,OPN,bicing,Pg Lluís Companys11,0.611111,alta disponibilidad


## guardamos en mongoDB

In [29]:
from pymongo import MongoClient
#mongodb conection
client = MongoClient('localhost', 27017)
db = client.bici
collection = db.datacleaned
collection.insert_many(data_mad.to_dict("records"))
collection.insert_many(data_bcn.to_dict("records"))